# Assistant-01<br/>The OpenAI reference demo

Reference:
- https://platform.openai.com/docs/assistants/overview

### Get an OpenAI client

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import common
import time

## Get an OpenAI client
client = common.get_openai_client()

### Create an assistant

In [2]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo-test"
)
assistant_id = assistant.id

### Create an assistant thread

In [3]:
thread = client.beta.threads.create()
thread_id = thread.id
thread_id

'thread_pKyOd8o2eJFRz4SfysqRJBbv'

### Create a user message in the assistant thread

In [4]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
message

ThreadMessage(id='msg_tFBStpr139kLwArJdb88EvCl', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1705035659, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_pKyOd8o2eJFRz4SfysqRJBbv')

## Run the assistant in the assistant thread

In [5]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

### Wait until the run is completed

In [6]:
while True:
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
  if run.status == "completed":
    break
  # TODO: Is this best practice?
  time.sleep(1)

### Get the messages for the thread and print them

In [7]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
for msg in messages:
  #print(msg)
  content = msg.content
  for line in content:
    txt = line.text
    # TODO: Is there a better way to do this? Should it be sorted by date?
    print(msg.role,txt.value)
  

assistant To solve the equation \(3x + 11 = 14\), we can rearrange the terms to isolate \(x\):

\[
3x = 14 - 11
\]
\[
3x = 3
\]
\[
x = \frac{3}{3}
\]
\[
x = 1
\]

So, the solution to the equation is \(x = 1\). If you have any more questions or need further assistance, feel free to ask!
user I need to solve the equation `3x + 11 = 14`. Can you help me?


### List all running assistants in the model

In [8]:
# List the registered assistants
assistants = client.beta.assistants.list()
for assistant in assistants.data:
    print(assistant)

# TODO: Do the assistans in the model and the threads, that have are not not disposed
# have a time limit or does there have to be a cleanup?


Assistant(id='asst_nmVH9gWfbCACxxtM8H2U7zWI', created_at=1705035658, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt-4-turbo-test', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])
Assistant(id='asst_dH8MRTuqwplfRBEDrnLJLjre', created_at=1705034667, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt-4-turbo-test', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])
Assistant(id='asst_iZxyZmci2o3WqxE7oDSwALpF', created_at=1705034026, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt-4-turbo-test', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])
Assistant(id='asst_9F6ZsUPuV

### Cleanup: clean the assistant and the thread thread

In [9]:
# TODO: Should the assisntant live for the life of the application?
client.beta.assistants.delete(assistant_id=assistant_id)

# TODO: Should the thread live for the life of the execution?
client.beta.threads.delete(thread_id=thread_id)

ThreadDeleted(id='thread_pKyOd8o2eJFRz4SfysqRJBbv', deleted=True, object='thread.deleted')